## 교차 검증 단순화
- scikit-learn의 model_selection 모듈 내에 모델 검증관련 기능 활용
- 교차 검증 데이터기반 검증 결과 처리

### [1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
# 모듈 로딩
import pandas as pd

# 생선 데이터 준비
fishDF = pd.read_csv('../data/fish.csv')

# 붓꽃 데이터 준비
irisDF = pd.read_csv('../data/iris.csv')
irisDF

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### [2] 데이터 준비 => 피쳐 & 타겟 분리

In [4]:
# 타겟 : Weight, 피쳐 : Length, Diagonal, Height, Width
fish_targetSR = fishDF[fishDF.columns[1]]
fish_featureDF = fishDF[fishDF.columns[2:]]

fish_targetSR.head(2), fish_featureDF.head(2)

(0    242.0
 1    290.0
 Name: Weight, dtype: float64,
    Length  Diagonal  Height   Width
 0    25.4      30.0   11.52  4.0200
 1    26.3      31.2   12.48  4.3056)

In [5]:
# 타겟 : Weight, 피쳐 : Length, Diagonal, Height, Width
iris_targetSR = irisDF.species
iris_featureDF = irisDF[irisDF.columns[:-1]]

iris_targetSR.head(2), iris_featureDF.head(2)

(0    setosa
 1    setosa
 Name: species, dtype: object,
    sepal_length  sepal_width  petal_length  petal_width
 0           5.1          3.5           1.4          0.2
 1           4.9          3.0           1.4          0.2)

## [3] 데이터 전처리 <hr>
- 피쳐 스케일링

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [7]:
# 학습용 테스트용 데이터 분리
from sklearn.model_selection import train_test_split

# 생선 데이터 => 학습용 데이터셋, 테스트용 데이터셋 => 회귀
fish_X_train, fish_x_test, fish_y_train, fish_y_test = train_test_split(fish_featureDF,
                                                                        fish_targetSR,
                                                                        test_size=0.2,
                                                                        random_state=5)

# 붓꽃 데이터 => 학습용 데이터셋, 테스트용 데이터셋 => 분류
iris_X_train, iris_x_test, iris_y_train, iris_y_test = train_test_split(iris_featureDF,
                                                                        iris_targetSR,
                                                                        test_size=0.2,
                                                                        stratify=iris_targetSR, #분류에서만
                                                                        random_state=5)

In [14]:
from sklearn.preprocessing import StandardScaler

fish_Scaler = StandardScaler()
fish_Scaler.fit(fish_X_train)
scaled_fish_X_train = fish_Scaler.transform(fish_X_train)
scaled_fish_X_test = fish_Scaler.transform(fish_x_test)

iris_Scaler = StandardScaler()
iris_Scaler.fit(iris_X_train)
scaled_iris_X_train = iris_Scaler.transform(iris_X_train)
scaled_iris_X_test = iris_Scaler.transform(iris_x_test)

### [4] 학습 <hr>

#### [4-1] 생선 무게 예측 모델

In [16]:
# 모듈 로딩
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

In [19]:
# 교차 검증으로 학습 진행
# => 준비 : 모델 인스턴스, 학습용 피쳐 데이터, 학습용 라벨 데이터
lr_model = LinearRegression()
# lr_model.fit(scaled_fish_X_train, fish_y_train)
# lr_model.fit(scaled_fish_X_test, fish_y_test)


In [35]:
# 학습/검증에 대한 평가 모든 결과에 대한 처리
result = cross_validate(lr_model, 
                        scaled_fish_X_train, 
                        fish_y_train, 
                        scoring=('accuracy', 'f1', 'r2', 'precision', 'recall', 'neg_mean_squared_error', 'neg_mean_absolute_error'),
                        return_train_score=True, 
                        return_estimator=True)
result

{'fit_time': array([0.00070596, 0.00088716, 0.00103307, 0.00049901, 0.000386  ]),
 'score_time': array([0.00063801, 0.00053287, 0.00086403, 0.00040126, 0.00037217]),
 'estimator': [LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression()],
 'test_r2': array([0.92104683, 0.84385378, 0.88592423, 0.64671954, 0.79031905]),
 'train_r2': array([0.87426416, 0.88779401, 0.88061108, 0.90297504, 0.89833592]),
 'test_neg_mean_squared_error': array([ -8767.84902315, -17815.75093903, -12344.87825138, -22006.47049028,
        -39450.52608702]),
 'train_neg_mean_squared_error': array([-16078.44783606, -13972.57866943, -15268.42472495, -13223.98109532,
        -10586.01039978])}

In [36]:
resultDF = pd.DataFrame(result)
resultDF

,fit_time,score_time,estimator,test_r2,train_r2,test_neg_mean_squared_error,train_neg_mean_squared_error
0,0.000706,0.000638,LinearRegression(),0.921047,0.874264,-8767.849023,-16078.447836
1,0.000887,0.000533,LinearRegression(),0.843854,0.887794,-17815.750939,-13972.578669
2,0.001033,0.000864,LinearRegression(),0.885924,0.880611,-12344.878251,-15268.424725
3,0.000499,0.000401,LinearRegression(),0.646720,0.902975,-22006.470490,-13223.981095
4,0.000386,0.000372,LinearRegression(),0.790319,0.898336,-39450.526087,-10586.010400


In [37]:
best_model = resultDF.iloc[0]['estimator']
best_model.coef_, best_model.intercept_

(array([ 373.98470744, -159.77931033,   90.53431501,   50.22123874]),
 408.522509249702)

In [53]:
### CV에 score만 추출
cross_val_score(lr_model, scaled_fish_X_train, fish_y_train)

array([0.92104683, 0.84385378, 0.88592423, 0.64671954, 0.79031905])

In [54]:
### CV에 predict만 추출
cross_val_predict(lr_model, scaled_fish_X_train, fish_y_train)


array([ 9.09792517e+01,  9.85612151e+01,  3.87029719e+02,  1.13011547e+02,
        6.81676563e+02,  2.82456988e+02,  5.34379642e+02,  3.61848302e+02,
        6.12934598e+02,  1.70756130e+02,  5.53222970e+02,  1.69433076e+01,
       -2.53895688e+01,  8.14926155e+02,  6.97225129e+01,  3.38157931e+02,
        4.76306355e+02,  7.67659158e+02,  6.55686457e+02,  1.80300946e+02,
        8.45315559e+02,  2.92145322e+02,  6.08539351e+02,  9.02782406e+02,
        6.99788981e+02,  9.40316876e+02,  7.47628344e+02,  3.28419355e+02,
        7.89622699e+02,  9.09130831e+02, -1.98986854e+02,  1.81089559e+02,
        6.36731679e+02, -1.09209894e+02,  3.57087822e+02,  7.88250361e+02,
        3.25180589e+02,  6.56473977e+02, -2.37032025e+02,  4.55882834e+01,
        9.57130255e+01, -2.10830505e+02,  1.28969696e+02, -2.21199132e+02,
       -1.10282630e+02,  6.39911566e+02,  2.12288357e+02,  2.41098815e+02,
        2.61932359e+02, -2.58301758e+02,  2.93250859e+01,  8.87950700e+02,
        2.46460034e+02,  

##  교차검증과 튜닝까지 한꺼번에 진행
- 단점 : 시간이 오래 걸림

In [55]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [70]:
from sklearn.linear_model import LogisticRegression
est=LogisticRegression(max_iter=10000, solver='liblinear')
params = {'penalty': ['l1', 'l2'],
          # 'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
          }

In [71]:
gscv = GridSearchCV(est, param_grid=params, return_train_score=True)

gscv.fit(scaled_iris_X_train, iris_y_train)

GridSearchCV(estimator=LogisticRegression(max_iter=10000, solver='liblinear'),
             param_grid={'penalty': ['l1', 'l2']}, return_train_score=True)

In [72]:
pd.DataFrame(gscv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000897,0.000164,0.000300,0.000072,l1,{'penalty': 'l1'},0.958333,0.958333,0.916667,0.916667,...,0.925,0.031180,1,0.958333,0.958333,0.958333,0.937500,0.958333,0.954167,0.008333
1,0.000526,0.000046,0.000255,0.000078,l2,{'penalty': 'l2'},0.875000,0.916667,0.916667,0.833333,...,0.875,0.037268,2,0.916667,0.916667,0.937500,0.947917,0.916667,0.927083,0.013176


In [73]:
gscv.best_estimator_, gscv.best_index_, gscv.best_params_, gscv.best_score_

(LogisticRegression(max_iter=10000, penalty='l1', solver='liblinear'),
 0,
 {'penalty': 'l1'},
 0.925)

#### 데이터에 적합한 모델 찾기

In [75]:
from sklearn.utils.discovery import all_estimators

In [88]:
all_estimators("classifier")

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('ClassifierChain', sklearn.multioutput.ClassifierChain),
 ('ComplementNB', sklearn.naive_bayes.ComplementNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('GaussianProcessClassifier',
  sklearn.gaussian_process._gpc.GaussianProcessClassifier),
 ('GradientBoostingClassifier',
  sklearn.ensemble._gb.GradientBoostingClassifier),
 ('HistGradientBoostingClassifier',
  sklearn.ensemble._hist_gradi

In [89]:
all_estimators("regressor")

[('ARDRegression', sklearn.linear_model._bayes.ARDRegression),
 ('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
 ('CCA', sklearn.cross_decomposition._pls.CCA),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('GammaRegressor', sklearn.linear_model._glm.glm.GammaRegressor),
 ('GaussianProcessRegressor',
  sklearn.gaussian_process._gpr.GaussianProcessRegressor),
 ('GradientBoostingRegressor', sklearn.ensemble._gb.GradientBoostingRegressor),
 ('HistGradientBoostingRegressor',
  sk

In [100]:
models = all_estimators("classifier")

for model_name, model in models:
    try:
        print(model().fit(scaled_iris_X_train, iris_y_train))
        # print(model().score(scaled_iris_X_test, iris_y_test))
    except Exception as e:
        print(e)

AdaBoostClassifier()
BaggingClassifier()
BernoulliNB()
CalibratedClassifierCV()
Negative values in data passed to CategoricalNB (input X)
__init__() missing 1 required positional argument: 'base_estimator'
Negative values in data passed to ComplementNB (input X)
DecisionTreeClassifier()
DummyClassifier()
ExtraTreeClassifier()
ExtraTreesClassifier()
GaussianNB()
GaussianProcessClassifier()
GradientBoostingClassifier()
HistGradientBoostingClassifier()
KNeighborsClassifier()
LabelPropagation()
LabelSpreading()
LinearDiscriminantAnalysis()
LinearSVC()
LogisticRegression()
LogisticRegressionCV()
MLPClassifier()
__init__() missing 1 required positional argument: 'estimator'
Negative values in data passed to MultinomialNB (input X)
NearestCentroid()
NuSVC()
__init__() missing 1 required positional argument: 'estimator'
__init__() missing 1 required positional argument: 'estimator'
__init__() missing 1 required positional argument: 'estimator'
PassiveAggressiveClassifier()
Perceptron()
Quadrat

/Users/sml/anaconda3/envs/EXAM_ML/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/sml/anaconda3/envs/EXAM_ML/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/sml/anaconda3/envs/EXAM_ML/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization h

SGDClassifier()
SVC()
__init__() missing 1 required positional argument: 'estimators'
__init__() missing 1 required positional argument: 'estimators'


In [103]:
all_estimators("classifier")


[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('ClassifierChain', sklearn.multioutput.ClassifierChain),
 ('ComplementNB', sklearn.naive_bayes.ComplementNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('GaussianProcessClassifier',
  sklearn.gaussian_process._gpc.GaussianProcessClassifier),
 ('GradientBoostingClassifier',
  sklearn.ensemble._gb.GradientBoostingClassifier),
 ('HistGradientBoostingClassifier',
  sklearn.ensemble._hist_gradi